## 11.1.2 preparation

In [ ]:
import sys
%matplotlib inline

In [ ]:
sys.path.append('../scripts/')

In [ ]:
from dp_policy_agent import *

In [ ]:
class QAgent(DpPolicyAgent):
    '''
    Initial version
    '''
    def __init__(self, time_interval, estimator, goal, puddle_coeff=100, widths=np.array([0.2,0.2,math.pi/18]).T,\
                lowerleft=np.array([-4,-4]).T, upperright=np.array([4,4]).T):
        super().__init__(time_interval, estimator, goal, puddle_coeff, widths, lowerleft, upperright)

In [ ]:
def trial(): 
    time_interval = 0.1
    world = PuddleWorld(400000, time_interval, debug=False)  #長時間アニメーション時間をとる

    ## 地図を生成して3つランドマークを追加 ##
    m = Map()
    for ln in [(-4,2), (2,-3), (4,4), (-4,-4)]: m.append_landmark(Landmark(*ln))
    world.append(m)   

    ##ゴールの追加##
    goal = Goal(-3,-3) 
    world.append(goal)
    
    ##水たまりの追加##
    world.append(Puddle((-2, 0), (0, 2), 0.1)) 
    world.append(Puddle((-0.5, -2), (2.5, 1), 0.1)) 

    ##ロボットを1台登場させる##
    init_pose = np.array([3, 3, 0]).T
    kf = KalmanFilter(m, init_pose)
    a = QAgent(time_interval, kf, goal)
    r = Robot(init_pose, sensor=Camera(m, distance_bias_rate_stddev=0, direction_bias_stddev=0), 
              agent=a, color="red", bias_rate_stds=(0,0))
    world.append(r)
    
    world.draw()
    return a

In [ ]:
a = trial()

## 11.1.3 set up Q

In [ ]:
class StateInfo:
    '''
    class for keeping Q values
    '''
    def __init__(self, action_num=3):
        self.q = np.zeros(action_num)
    
    def greedy(self):
        return np.argmax(self.q)
    
    def pi(self):
        return self.greedy()

In [ ]:
class QAgent(DpPolicyAgent):
    '''
    Second version
    '''
    def __init__(self, time_interval, estimator, puddle_coeff=100, widths=np.array([0.2,0.2,math.pi/18]).T,\
                lowerleft=np.array([-4,-4]).T, upperright=np.array([4,4]).T):
        super().__init__(time_interval, estimator, None, puddle_coeff, widths, lowerleft, upperright)
        
        #const
        self.smallval = 0.1 # initial value is subtracted with this value in case the action is not in policy.

        nx, ny, nt = self.index_nums # in DpPolicyAgent. nx is number of x indices.
        self.indexes = list(itertools.product(range(nx), range(ny), range(nt)))
        self.actions = list(set([tuple(self.policy_data[i]) for i in self.indexes])) # in this example, actions=[forward,turn]
        self.statespace = self.set_action_value_function() 
        
    def set_action_value_function(self, value_file="../section_reinforcement_learning/puddle_ignore_values.txt"):
        statespace = {}
        for line in open(value_file, 'r'):
            d = line.split()
            index= (int(d[0]), int(d[1]), int(d[2]))
            value = float(d[3])
            statespace[index] = StateInfo(len(self.actions)) # init state space
            
            for i, a in enumerate(self.actions):
                statespace[index].q[i] = value if tuple(self.policy_data[index]) == a\
                else value - self.smallval
            
        return statespace
    
    def policy(self, pose, goal=None): # we need goal argument to overload!
        index = self.to_index(pose, self.pose_min, self.index_nums, self.widths) # from DpPolicyAgent class
        a = self.statespace[tuple(index)].pi() # a is the index of the action.
        return self.actions[a]

In [ ]:
def trial(): 
    time_interval = 0.1
    world = PuddleWorld(400000, time_interval, debug=False)  #長時間アニメーション時間をとる

    ## 地図を生成して3つランドマークを追加 ##
    m = Map()
    for ln in [(-4,2), (2,-3), (4,4), (-4,-4)]: m.append_landmark(Landmark(*ln))
    world.append(m)   

    ##ゴールの追加##
    goal = Goal(-3,-3) 
    world.append(goal)
    
    ##水たまりの追加##
    world.append(Puddle((-2, 0), (0, 2), 0.1)) 
    world.append(Puddle((-0.5, -2), (2.5, 1), 0.1)) 

    ##ロボットを1台登場させる##
    init_pose = np.array([3, 3, 0]).T
    kf = KalmanFilter(m, init_pose)
    a = QAgent(time_interval, kf)
    r = Robot(init_pose, sensor=Camera(m, distance_bias_rate_stddev=0, direction_bias_stddev=0), 
              agent=a, color="red", bias_rate_stds=(0,0))
    world.append(r)
    
    world.draw()
    return a

In [ ]:
trial()

## 11.1.4 espilon-greedy

In [ ]:
class StateInfo:
    '''
    class for keeping Q values
    version2.
    '''
    def __init__(self, action_num=3, epsilon=0.7):
        self.q = np.zeros(action_num)
        self.epsilon = epsilon
    
    def greedy(self):
        return np.argmax(self.q)
    
    def epsilon_greedy(self, epsilon):
        if random.random() < epsilon:
            return random.choice(range(len(self.q)))
        else:
            return self.greedy()
    
    def pi(self):
        return self.epsilon_greedy(self.epsilon)

In [ ]:
trial()

## 11.1.5 update Q function

In [ ]:
class StateInfo:
    '''
    class for keeping Q values
    version2.
    '''
    def __init__(self, action_num=3, epsilon=0.7):
        self.q = np.zeros(action_num)
        self.epsilon = epsilon
    
    def greedy(self):
        return np.argmax(self.q)
    
    def epsilon_greedy(self, epsilon):
        if random.random() < epsilon:
            return random.choice(range(len(self.q)))
        else:
            return self.greedy()
    
    def pi(self):
        return self.epsilon_greedy(self.epsilon)
    
    def max_q(self):
        return max(self.q)

In [ ]:
class QAgent(DpPolicyAgent):
    '''
    Third version
    '''
    def __init__(self, time_interval, estimator, puddle_coeff=100, alpha=0.5,\
                 widths=np.array([0.2,0.2,math.pi/18]).T,\
                lowerleft=np.array([-4,-4]).T, upperright=np.array([4,4]).T):
        super().__init__(time_interval, estimator, None, puddle_coeff, widths, lowerleft, upperright)
        
        #const
        self.smallval = 0.1 # initial value is subtracted with this value in case the action is not in policy.

        nx, ny, nt = self.index_nums # in DpPolicyAgent. nx is number of x indices.
        self.indexes = list(itertools.product(range(nx), range(ny), range(nt)))
        self.actions = list(set([tuple(self.policy_data[i]) for i in self.indexes])) # in this example, actions=[forward,turn]
        self.statespace = self.set_action_value_function() 
        self.alpha = alpha # weight of new value for update Q 
        self.s = None
        self.a = None
        self.update_end = False # whether Q update is finished or not.
        
        
    def set_action_value_function(self, value_file="../section_reinforcement_learning/puddle_ignore_values.txt"):
        statespace = {}
        for line in open(value_file, 'r'):
            d = line.split()
            index= (int(d[0]), int(d[1]), int(d[2]))
            value = float(d[3])
            statespace[index] = StateInfo(len(self.actions)) # init state space
            
            for i, a in enumerate(self.actions):
                statespace[index].q[i] = value if tuple(self.policy_data[index]) == a\
                else value - self.smallval
            
        return statespace
    
    
    def policy(self, pose, goal=None): # we need goal argument to overload!
        index = self.to_index(pose, self.pose_min, self.index_nums, self.widths) # from DpPolicyAgent class
        s = tuple(index)
        a = self.statespace[s].pi() # a is the index of the action.
        return s,a
    
    
    def decision(self, observation=None):
        if self.update_end: return 0.0, 0.0
        if self.in_goal: self.update_end = True
            
        # observe self position by Kalman filter
        self.estimator.motion_update(self.prev_nu, self.prev_omega, self.time_interval)
        self.estimator.observation_update(observation)
        
        # deside action
        next_s, next_a = self.policy(self.estimator.pose)
        r = self.time_interval * self.reward_per_sec()
        self.total_reward += r
        
        # Q learning
        self.q_update(self.s, self.a, r, next_s)
        self.s = next_s
        self.a = next_a
        
        # output velocities
        self.prev_nu, self.prev_omega = self.actions[next_a]
        return self.actions[next_a]
    
    
    def q_update(self, s, a, r, next_s):
        if s == None: return
        
        q = self.statespace[s].q[a]
        next_q = self.final_value if self.in_goal else self.statespace[next_s].max_q()
        self.statespace[s].q[a] = (1 - self.alpha) * q + self.alpha * (r + next_q)
        
#         # log
#         with open("log.txt", "a") as f:
#             f.write("{} {} {} prev_q:{:.2f}, next_step_max_q:{:.2f}, new_q:{:.2f}\n"\
#                     .format(s, r, next_s, q, next_q, self.statespace[s].q[a]))
        

In [ ]:
trial()

## 11.1.6 recurse episodes

In [ ]:
import copy

In [ ]:
class WarpRobot(Robot):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.init_agent = copy.deepcopy(self.agent)
        
    def choose_pose(self):
        xy = random.random()*6 - 2 #x or y. around the center:4
        t = random.random() * 2*math.pi
        return np.array([3, xy, t].T) if random.random()>0.5 else np.array([xy, 3, t]).T
    
    def reset(self):
        # reset agent
        tmp_ss = self.agent.statespace
        self.agent = copy.deepcopy(self.init_agent)
        self.agent.statespace = tmp_ss
        
        # set initial position
        self.pose = self.choose_pose()
        self.agent.estimator.belief = multivariate_normal(mean=self.pose, cov=np.diag([1e-10,1e-10,1e-10]))
        
        # delete black tracking lines
        self.poses = []
        
    def one_step(self, time_interval):
        if self.agent.update_end:
            with open("log.txt","a") as f:
                f.write("{}\n".format(self.agent.total_reward + self.agent.final_value))
            self.reset()
            return
        super().one_step(time_interval)

In [ ]:
def trial(): 
    time_interval = 0.1
    world = PuddleWorld(400000, time_interval, debug=False)  #長時間アニメーション時間をとる

    ## 地図を生成して3つランドマークを追加 ##
    m = Map()
    for ln in [(-4,2), (2,-3), (4,4), (-4,-4)]: m.append_landmark(Landmark(*ln))
    world.append(m)   

    ##ゴールの追加##
    goal = Goal(-3,-3) 
    world.append(goal)
    
    ##水たまりの追加##
    world.append(Puddle((-2, 0), (0, 2), 0.1)) 
    world.append(Puddle((-0.5, -2), (2.5, 1), 0.1)) 

    ##ロボットを1台登場させる##
    init_pose = np.array([3, 3, 0]).T
    kf = KalmanFilter(m, init_pose)
    a = QAgent(time_interval, kf)
    r = WarpRobot(init_pose, sensor=Camera(m, distance_bias_rate_stddev=0, direction_bias_stddev=0), 
              agent=a, color="red", bias_rate_stds=(0,0))
    world.append(r)
    
    world.draw()
    return a

In [ ]:
trial()